## 函数的参数引用
>Python 唯一支持的参数传递模式是共享传参（ call by sharing）, 共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是说，函数内部的形参是实参的别名。

In [1]:
def func(a, b):
    a += b
    return a

a = [1, 2]
b = [3, 4]
print("执行函数结果:{}".format(func(a, b)))
print("a:{0}, b:{1}".format(a, b))

执行函数结果:[1, 2, 3, 4]
a:[1, 2, 3, 4], b:[3, 4]


>列表是可变对象, 当作参数传入函数引用时会被修改, 同时实参 `a` 和形参 `para1` 指向的是同一个列表对象, 对象内容被修改了, 相当于是直接修改了这个对象的内容, 所以实参输出结果也变了
![](images/para1.png)

In [2]:
t = (10, 20)
u = (20, 30)
print("执行函数结果:{}".format(func(t, u)))
print("t:{0}, u:{1}".format(t,u))

执行函数结果:(10, 20, 20, 30)
t:(10, 20), u:(20, 30)


In [3]:
x = 1
y = 2
print("执行函数结果:{}".format(func(x, y)))
print("x:{0}, y:{1}".format(x, y))

执行函数结果:3
x:1, y:2


>如图, 一开始实参 `t` 和形参 `para1` 指向的是同一个对象, 但是因为在Python中元祖是不可变对象, 所以当形参内容执行函数后被修改了, 就会创建一个新的元祖对象, 形参指向的就是这个对象, 而不是原来跟实参指向一致了
![](images/para3.png)
![](images/para2.png)

## 不要使用可变类型作为参数默认值

> 在Python中我们可以将对象作为参数传入函数, 同时还能给函数的参数设置默认值, 但是应该⚠️注意不能将可变类型作为参数默认值, 比如下面示例:

In [4]:
class HauntedBus:
    
    def __init__(self, passengers=[]):
        """这里passengers设置默认参数为一个空列表,这是错误的"""
        self.passengers = passengers 
        
    def pick(self, name):
        self.passengers.append(name) 
        
    def drop(self, name):
        self.passengers.remove(name)
        

In [5]:
bus1 = HauntedBus(['A1', 'B1'])
print("初始化 bus1 :{0}".format(bus1.passengers))
bus1.pick('C1')
bus1.drop('B1')
print("修改后 bus1 :{0}".format(bus1.passengers))
bus2 = HauntedBus()
print("初始化 bus2 :{0}".format(bus2.passengers))
bus2.pick('D1')
print("修改后 bus2 :{0}".format(bus2.passengers))
bus3 = HauntedBus()
print("初始化 bus3 :{0}".format(bus3.passengers))

初始化 bus1 :['A1', 'B1']
修改后 bus1 :['A1', 'C1']
初始化 bus2 :[]
修改后 bus2 :['D1']
初始化 bus3 :['D1']


>可以看到初始化 `bus3` 后其默认赋值不是空列表了, 而是修改后`bus2`内容

### 再看: 

In [6]:
bus3.pick('E1')
print("修改后 bus2 :{0}".format(bus2.passengers))
print(bus2.passengers is bus3.passengers)
print("修改后 bus1 :{0}".format(bus1.passengers))

修改后 bus2 :['D1', 'E1']
True
修改后 bus1 :['A1', 'C1']


> 从以上例子可以看出, `bus2.passengers`, `bus3.passengers` 指代同一个列表, 但 `bus1.passengers`不是<p>
>So why?<p>
> 因为初始化 `bus1`时 指定了一个初始列表, 而 `bus2` 和 `bus3`都是默认为空, 所以它们共享同一个列表<p>
如图所示:
![](images/para4.png)
>出现这个问题的根源是，默认值在定义函数时计算（通常在加载模块时），因此默认值变成了函数对象的属性, 
>比如这里就是因为 `self.passengers` 变成了 `passengers` 参数默认值的别名。
>因此，如果默认值是可变对象，而且修改了它的值，那么后续的函数调用都会受到影响。

### 再看下面这样定义, 使用 `None`  作为接收可变值的参数的默认值

In [8]:
class Bus:
    
    def __init__(self, passengers=None):        # 参考这样初始化
        if passengers is None:
            self.passengers = []        
        else:
            self.passengers = passengers   # ⚠️注意这里
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)
        

team = ['A1', 'B1', 'C1', 'D1', 'E1']
bus4 = Bus(team)
bus4.drop('A1')
bus4.drop('B1')
print(team)

['C1', 'D1', 'E1']


> 看结果, 修改了 `bus4`, 为什么 `team`这个列表对象内容也发生变化了呢?<p>
> 注意上面标记处, `self.passengers = passengers`, 因为`passengers`是列表可变对象, 而`passengers` 和`team`指向的是同一个列表对象, 所以内容发生了变化, 参考上面`函数的参数引用`说明<p>
>正确的做法应该是如[`copy_principle.ipynb`](copy_principle.ipynb)中那样定义一个 `bus` 类, 修改为`self.passengers = list(passengers)`<p>